In [194]:
import os
import pandas as pd

In [195]:
df = pd.read_excel("/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels/Labelling_Prateek_Guillermo.xlsx")
# df = pd.read_excel("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/Labels/Labelling_Prateek_Guillermo.xlsx")

In [196]:
def code_preprocessing(file):
    with open(file) as dataset_obj:
        codeLines = dataset_obj.read()
    comment_lines = []
    raw_codeLines = codeLines.replace("\t","").split("\n")
    multi_line_flag = 0
    for line_number in range(len(raw_codeLines)):
        if multi_line_flag:
            if raw_codeLines[line_number].__contains__("*/"):
                multi_line_flag = 0
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("/*"):
            if raw_codeLines[line_number].startswith("/*") and not(raw_codeLines[line_number].__contains__("*/")):
                comment_lines.append(line_number)
                multi_line_flag = 1
            elif raw_codeLines[line_number].__contains__("/*") and not(raw_codeLines[line_number].startswith("/*")):
                if raw_codeLines[line_number].__contains__("*/"):
                    psuedo_multi_line_start = raw_codeLines[line_number].find("/*")
                    psuedo_multi_line_end = raw_codeLines[line_number].find("*/")
                    temporary_line = raw_codeLines[line_number][:psuedo_multi_line_start] + raw_codeLines[line_number][psuedo_multi_line_end+2:]
                    raw_codeLines[line_number] = temporary_line

        elif raw_codeLines[line_number].startswith("//"):
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("//"):
            comment_start = raw_codeLines[line_number].find('//')
            raw_codeLines[line_number] = raw_codeLines[line_number][:comment_start]
    
    

    comment_lines.reverse()
    for i in comment_lines:
        raw_codeLines.pop(i)

    def insert_space(string, index):
        string_copy = ""
        for i in range(len(string)):
            if i==(index):
                string_copy += " "
                string_copy += string[i]
                string_copy += " "
                continue
            string_copy += string[i]
        return string_copy

    def find_char_indices(input_string, char):
        indices = []
        replacement_token = 0
        for index, character in enumerate(input_string):
            if character == char:
                indices.append(index + 2*replacement_token)
                replacement_token+=1
        return indices

    def space_out(string, char):
        indices = find_char_indices(string, char)

        for i in indices:
            string = insert_space(string, i)
        return string

    # for line_number in range(len(raw_codeLines)):
    #     placeHolder = raw_codeLines[line_number]
    #     placeHolder = space_out(placeHolder, ";")
    #     placeHolder = space_out(placeHolder, "(")
    #     placeHolder = space_out(placeHolder, ")")
    #     placeHolder = space_out(placeHolder, ",")
    #     if placeHolder.endswith(";"):
    #         raw_codeLines[line_number] = "<start> " + placeHolder.replace(";","<end>")
    #     elif not(placeHolder.endswith(";")):
    #         raw_codeLines[line_number] = "<start> " + placeHolder + " <end>"

    return raw_codeLines


In [197]:
def comment_finder(file):
    with open(file) as dataset_obj:
        codeLines = dataset_obj.read()
        comment_lines = []
        raw_codeLines = codeLines.replace("\t","").split("\n")
        multi_line_flag = 0
        for line_number in range(len(raw_codeLines)):
            if multi_line_flag:
                if raw_codeLines[line_number].__contains__("*/"):
                    multi_line_flag = 0
                comment_lines.append(line_number)
            elif raw_codeLines[line_number].__contains__("/*"):
                if raw_codeLines[line_number].startswith("/*") and not(raw_codeLines[line_number].__contains__("*/")):
                    comment_lines.append(line_number)
                    multi_line_flag = 1
                elif raw_codeLines[line_number].__contains__("/*") and not(raw_codeLines[line_number].startswith("/*")):
                    if raw_codeLines[line_number].__contains__("*/"):
                        psuedo_multi_line_start = raw_codeLines[line_number].find("/*")
                        psuedo_multi_line_end = raw_codeLines[line_number].find("*/")
                        temporary_line = raw_codeLines[line_number][:psuedo_multi_line_start] + raw_codeLines[line_number][psuedo_multi_line_end+2:]
                        raw_codeLines[line_number] = temporary_line

            elif raw_codeLines[line_number].startswith("//"):
                comment_lines.append(line_number)
            elif raw_codeLines[line_number].__contains__("//"):
                comment_start = raw_codeLines[line_number].find('//')
                raw_codeLines[line_number] = raw_codeLines[line_number][:comment_start]
    return comment_lines

In [198]:
file_start = {}
for i in range(len(df['File'])):
    if pd.isnull(df['File'][i]) == False:
        file_start[df['File'][i]] = i

BOTTOM_LINE = file_start['TOTAL NO. OF FILES']
file_list = list(file_start.keys())[:-1]
file_vulnerabilities = {}

for i in range(len(file_list)-1):
    vulnerable_lines=[]
    for j in range(file_start[file_list[i]],file_start[file_list[i+1]]-1):
        if (df['Comment'][j].split('|')[1].split()[0]) == "BRANCH:":
            if (pd.isna(df.at[j,'False Positive'])):
                vulnerable_lines.append(int(df['Comment'][j].split('|')[0].split()[1]))
        if (not(pd.isna(df.at[j,'Lines Missed']))):
            if (not(pd.isna(df.at[j,'Branch.1']))):
                line = df['Lines Missed'][j].split()
                if (not(line[0].isnumeric())):
                    vulnerable_lines.append(int(line[1].strip(":")))
                else:
                    vulnerable_lines.append(int(line[0].strip(":")))
    vulnerable_lines.sort()

    file_vulnerabilities[file_list[i]]=vulnerable_lines

file_list = file_list[:-1]

vulnerable line adjustment

In [199]:
os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach/dataset")
# os.chdir("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/dataset")
for file in file_list:
    comments = comment_finder(file)
    for vul_line in range(len(file_vulnerabilities[file])):
        for num in range(len(comments)):
            if num == len(comments)-1:
                (file_vulnerabilities[file][vul_line]) = ((file_vulnerabilities[file][vul_line])-(len(comments)))
            elif comments[num] < (file_vulnerabilities[file][vul_line]):
                None
            elif comments[num] > (file_vulnerabilities[file][vul_line]):
                (file_vulnerabilities[file][vul_line]) = ((file_vulnerabilities[file][vul_line])-(num+1))
                break


In [200]:
labeled_dataset = {'Lines':[], 'Label':[]}
labeled_dataset = pd.DataFrame(labeled_dataset)
labeled_dataset

for file in file_list:
    code = code_preprocessing(file)
    for line in range(len(code)):
        if (line) in file_vulnerabilities[file]:
            data = {'Lines': code[line], 'Label': 'Insecure'}
        else:
            data = {'Lines': code[line], 'Label': 'Secure'}
        labeled_dataset.loc[len(labeled_dataset)] = data

In [201]:
raw_code = list(labeled_dataset['Lines'])

In [202]:
raw_code #corpus

['',
 '',
 '#include <Compiler.h>',
 '#include <TimeDelay.h>',
 '#include "HardwareProfile.h"',
 '#include "Peripherals/Nand.h"',
 '#include "Peripherals/Accel.h"',
 '#ifdef USE_GYRO',
 '#include "Peripherals/Gyro.h"',
 '#endif',
 '#include "Peripherals/Rtc.h"',
 '#include "USB/USB.h"',
 '#include "USB/usb_function_msd.h"',
 '#include "USB/usb_function_cdc.h"',
 '#include "MDD File System/FSIO.h"',
 '#include "Usb/USB_CDC_MSD.h"',
 '#include "Ftl/FsFtl.h"',
 '#include "Utils/FsUtils.h"',
 '#include "Utils/Fifo.h"',
 '#include "Utils/Util.h"',
 '#include "Peripherals/Analog.h"',
 '#include "Settings.h"',
 '#include "Logger.h"',
 '',
 '#include "AX3 SelfTest.h"',
 '',
 '',
 'void RunAttached(void);',
 'void RunLogging(void);',
 'void TimedTasks(void);',
 'void LedTasks(void);',
 'void RunTestSequence(void);',
 '',
 'void __attribute__((interrupt,auto_psv)) _DefaultInterrupt(void)',
 '{',
 ' static unsigned int INTCON1val;',
 'LED_SET(LED_MAGENTA);',
 'INTCON1val = INTCON1;',
 'Nop();',
 

In [203]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN

In [204]:
vectorizer = TfidfVectorizer()
transformed_output = vectorizer.fit_transform(raw_code)
print(len(vectorizer.vocabulary_.keys()))

2019


In [205]:
for i in transformed_output:
    print(i.shape)

(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)
(1, 2019)


In [206]:
labeled_dataset.shape

(5205, 2)

In [207]:
all_feature_names = vectorizer.get_feature_names_out()

for word in all_feature_names:
    
    #let's get the index in the vocabulary
    indx = vectorizer.vocabulary_.get(word)
    
    #get the score
    idf_score = vectorizer.idf_[indx]/10
    
    print(f"{word} : {idf_score}")

vectorizer.idf_[vectorizer.vocabulary_.get('start')]

00 : 0.8864419904994566
0000 : 0.8864419904994566
0000001 : 0.8864419904994566
00_resource_generation : 0.8864419904994566
01 : 0.8864419904994566
02x : 0.7611656936499197
08x : 0.7254981992560465
0b00001001 : 0.8864419904994566
0d100 : 0.8864419904994566
0x : 0.7254981992560465
0x0 : 0.5446693221381199
0x00 : 0.707266043576651
0x0000 : 0.7611656936499197
0x0000000f : 0.8458954796886401
0x001f : 0.8864419904994566
0x00ff : 0.715967181275614
0x01 : 0.817127272443462
0x02 : 0.8864419904994566
0x05 : 0.8864419904994566
0x06 : 0.8864419904994566
0x0640 : 0.8864419904994566
0x07e0 : 0.8864419904994566
0x07ff : 0.8864419904994566
0x0b : 0.8864419904994566
0x0d : 0.8864419904994566
0x0e : 0.8864419904994566
0x0f : 0.6724353741498295
0x1 : 0.7360342508218292
0x10 : 0.8458954796886401
0x10000ul : 0.8864419904994566
0x11 : 0.8864419904994566
0x113 : 0.8864419904994566
0x12 : 0.8864419904994566
0x1234 : 0.8864419904994566
0x12345678 : 0.8864419904994566
0x13 : 0.8864419904994566
0x14 : 0.88644199

8.864419904994566

In [216]:
all_feature_names[940]

'include'

In [209]:
labeled_dataset['Label']=labeled_dataset['Label'].map({"Secure" : 0, "Insecure":1})
x_train, x_test, y_train, y_test = train_test_split(labeled_dataset['Lines'], labeled_dataset['Label'], test_size=0.2, random_state=32, stratify=labeled_dataset['Label'])
x_train, x_test, y_train, y_test

(2099            case IOTX_HTTP_UPDATE_SESSION_ERROR:
 4185                                                
 4837                  set_wifi_led_state(blueValue);
 3749                                                
 4884                                               }
                             ...                     
 2754                                        return; 
 5156                                               {
 2128                                   goto do_exit;
 3211                                               }
 4098                                                
 Name: Lines, Length: 4164, dtype: object,
 1449                                                    }
 5201                                   TOGGLE_STATUS_LED;
 4947            qcloud_iot_explorer_demo(CONFIG_DEMO_E...
 395                  NOT_STARTED_NO_INTERVAL     = 3,    
 2703                                                     
                               ...                        
 2004    

In [210]:
# vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(labeled_dataset['Lines'])


In [211]:
labeled_dataset['Lines'].shape

(5205,)

In [212]:
print(X_train_tfidf.shape)

(5205, 2019)


In [214]:
print(X_train_tfidf[2])

  (0, 455)	0.887534545521301
  (0, 940)	0.4607411751800548
